# <p style="text-align: center;, font-style: strong;">Section 1.2 - Training Logistic Regression Model</p>
### <p style="text-align: center;">(Almond 0.9.1, Scala 2.12.10)</p>

## Dependencie

In [ ]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.0`
interp.load.ivy(coursierapi.Dependency.of("org.platanios", "tensorflow_2.12", "0.4.1").withClassifier("linux-cpu-x86_64"))

## Imports

In [ ]:
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

import java.awt.{ Image => JImage }
import java.io.{ ByteArrayOutputStream, ByteArrayInputStream}
import scala.io.Source
import java.awt.image.BufferedImage
import java.awt.{ Color => JColor }
import javax.imageio.ImageIO
import scala.util.Random
import java.util.Base64
import org.platanios.tensorflow.api.tf
import org.platanios.tensorflow.api.tensors.Tensor
import org.platanios.tensorflow.api.core.Shape
import org.platanios.tensorflow.api.core.client.{Session, FeedMap}

In [ ]:
def transformDataIntoTensors(pathData: String, widthImage: Int): (Tensor[Float], Tensor[Int]) = {
    val data = Source.fromFile(pathData).getLines.toList
    val numImages = data.length
    var dataNormalized: Array[Array[Double]] = Array.ofDim[Double](numImages, widthImage * widthImage)
    var labels: List[Int] = List()

    for ((line, index) <- data.zipWithIndex) {
        val splitLine = line.split(" ")
        val base64Decoded: Array[Byte] = Base64.getDecoder.decode(splitLine.head)
        val label = splitLine.last
        val bufferedImage = ImageIO.read(new ByteArrayInputStream(base64Decoded))
        labels = if (label == "green") 1 :: labels else 0 :: labels // 1 = green, 0 = blue
        for (x <- 0 until bufferedImage.getWidth) {
            for (y <- 0 until bufferedImage.getHeight) {
                val color = bufferedImage.getRGB(x, y)
                val green = ((color & 0x0000ff00) >> 8) / 255.0 // normalize
                dataNormalized(index)(x * widthImage + y) = green
            }
        }
    }

    val labelsTensor = Tensor(labels.reverse).reshape(Shape(numImages))
    val imagesTensor = Tensor(dataNormalized.flatten.toList).reshape(Shape(numImages, widthImage * widthImage)).toFloat // utiliser fill plus optimal je pense
    val tensorHotEncoded = tf.oneHot(labelsTensor, 2, 1, 0) // 2 = nb dimensions, 1 = on value, 0 = off value
    (imagesTensor, Session().run(fetches = tensorHotEncoded))
}

In [ ]:
// Transform BufferedImage to a bytes Array
def convertBufferedImgToByteArray(buffImg: BufferedImage): Array[Byte] = {
    val baos = new ByteArrayOutputStream()
    ImageIO.write(buffImg, "png", baos)
    baos.toByteArray()
}

def resize(img: Array[Byte], height: Int, width: Int): Array[Byte] = {
    val bais = new ByteArrayInputStream(img)
    val bufferedImg = ImageIO.read(bais)
    val newImg = bufferedImg.getScaledInstance(width, height, JImage.SCALE_SMOOTH)
    val resized = new BufferedImage(width, height, BufferedImage.TYPE_INT_RGB)
    val g2d = resized.createGraphics()
    g2d.drawImage(newImg, 0, 0, null)
    g2d.dispose()
    convertBufferedImgToByteArray(resized)
}

// Generate a new blue-green color 
// initialGreen = First green choice randomly
// step = step to choose the new green color
// rnd = random generator
// RETURN a new green and blue value with 
// (green: Int = random value between [intialGreen - step, initialGreen + step], blue: Int = 255 - green)
def generateRandomColor(initialGreen: Int, step: Int, rnd: Random): (Int, Int) = {
    // handle border cases
    val start = if (initialGreen - step < 0) 0 else initialGreen - step
    val end = if (initialGreen + step > 255) 255 else initialGreen + step
    val randomGreen = start + rnd.nextInt((end - start) + 1)
    // new color is [r = 0, g = randomGreen, b = 255 - randomGreen]
    (randomGreen, 255 - randomGreen)
}

// Generate a random image
// size of the image, size = width = height
// step = step to choose the new green color
// rnd = random generator
// RETURN random image: BufferedImage
def generateRandomImage(size: Int, step: Int, rnd: Random): BufferedImage = {
    val rand = new Random(System.nanoTime)
    val image = new BufferedImage(size, size, BufferedImage.TYPE_INT_RGB)
    val initialGreen = rnd.nextInt(256) // random green between [0, 255]
    val initialBlue = (255 - initialGreen)
    for (x <- 0 until image.getWidth) {
        for (y <- 0 until image.getHeight) {
            // fill other with color depending from the initialGreen and the step variable
            val (newGreen, newBlue) = generateRandomColor(initialGreen, step, rnd)
            image.setRGB(x, y, new JColor(0, newGreen, newBlue).getRGB)
        }
    }
    image
}

// Generate random image and convert it to base64 representation
// step = step to choose the new green color
// rnd = random generator
// RETURN String = base64 random image
def generateBase64Image(size: Int, step: Int, rnd: Random): String = {
    val buffImage: BufferedImage = generateRandomImage(size, step, rnd)
    val byteImage: Array[Byte] = convertBufferedImgToByteArray(buffImage)
    Base64.getEncoder.encodeToString(byteImage)
}

In [ ]:
// Visualize results
def showAnswers(nameFile: String) {
    for (line <- Source.fromFile(nameFile).getLines) {
        val splitLine = line.split(" ")
        val base64Decoded: Array[Byte] = Base64.getDecoder.decode(splitLine.head)
        val image = Image(resize(base64Decoded, 100, 100)).withFormat(Image.PNG).display
        val label = splitLine.last
        println(label)
    }
}

# TRAINING

### Parameters

In [ ]:
val learningRate = 0.35f
val epochs = 100 // number of `iteration` in this case
val widthImage = 5
val pathData = "../resources/labels.txt"
val step = 20

val classNumber = 2 // blue or green

### Visualize

In [ ]:
showAnswers(pathData)

### Model Creation
We use a softmax activation because we want multi-class classification (= blue or green and not both at the same time)

In [ ]:
val sess = Session()
val sizeImage = widthImage * widthImage // num features

val imagesPlaceholder = tf.placeholder[Float](Shape(-1, sizeImage)) // = x
val labelsPlaceholder = tf.placeholder[Float](Shape(-1, classNumber)) // = y_

val W = tf.variable[Float]("weights", Shape(sizeImage, classNumber), tf.ZerosInitializer)
val B = tf.variable[Float]("biais", Shape(classNumber), tf.ZerosInitializer)

val labelsInfered = tf.add(tf.matmul(imagesPlaceholder, W), B) // W.x + B = (a.x + b) ... = Ŷ

val cost = tf.mean(tf.softmaxCrossEntropy(logits = labelsInfered, labels = labelsPlaceholder))

val optimizer = tf.train.GradientDescent(learningRate = learningRate).minimize(cost) 

sess.run(targets = tf.globalVariablesInitializer())

### Training the model

In [ ]:
val (imagesTensor, labelsTensor) = transformDataIntoTensors(pathData, widthImage)
val correct_prediction = tf.mean(tf.equal(tf.argmax(input = labelsInfered, axes = -1, outputDataType = Int), tf.argmax(input = labelsPlaceholder, axes = -1, outputDataType = Int)).toFloat)

val history = (1 to epochs).map { e => 
    val (costIte, accIte) = sess.run(fetches = (cost, correct_prediction), targets = optimizer, feeds = FeedMap(Map(imagesPlaceholder -> imagesTensor, labelsPlaceholder -> labelsTensor.toFloat)))
    if (e % 10 == 0) { println(s"Train loss at iteration $e = ${costIte.scalar}\nAccuracy at iteration $e = ${accIte.scalar * 100}%\n") }
    (costIte.scalar, accIte.scalar)
}

### Display cost

In [ ]:
val weight = sess.run(fetches = W.value)
val biais  = sess.run(fetches = B.value)

In [ ]:
// If it doesn't display, remove checkpoints in the same folder and run all the notebook again
{{
val costHistory = Scatter(
    (1 to epochs), 
    history.map(h => h._1),
    name = "Cost history"
)

val accHistory = Scatter(
    (1 to epochs), 
    history.map(h => h._2),
    name = "Accuracy history"
)

val data = Seq(costHistory, accHistory)
val layout = Layout(title = "Cost and Accuracy history")
plot(data, layout)
}}

### Test our trained model

In [ ]:
val rand = new Random(System.nanoTime) // random generator for random images

for (i <- 0 to 20) {
    val newData: Array[Double] = Array.ofDim[Double](sizeImage)
    val img = generateBase64Image(widthImage, step, rand)
    val base64Decoded: Array[Byte] = Base64.getDecoder.decode(img)
    val bufferedImage = ImageIO.read(new ByteArrayInputStream(base64Decoded))
    val image = Image(resize(base64Decoded, 100, 100)).withFormat(Image.PNG).display

    for (x <- 0 until bufferedImage.getWidth) {
        for (y <- 0 until bufferedImage.getHeight) {
            val color = bufferedImage.getRGB(x, y)
            val green = ((color & 0x0000ff00) >> 8) / 255.0 // normalize
            newData(x * widthImage + y) = green
        }
    }


    val imagesTensor = Tensor(newData.toList).reshape(Shape(1, sizeImage))
    val labelDetected = sess.run(fetches = tf.argmax(tf.softmax(labelsInfered), -1, Int), feeds = FeedMap(Map(imagesPlaceholder -> imagesTensor.toFloat)))
    if (labelDetected.scalar == 1) println("green") else println("blue")
}